In [1]:
#######################################################
###                                                 ###
###   /!\  RUN DL_LoadProduct1.ipynb BEFORE   /!\   ###
###                                                 ###
#######################################################

In [2]:
library(jsonlite)
library(rlist)
library(data.table)
suppressPackageStartupMessages(library("dplyr"))
library(XML)
library(methods)

In [3]:
# We load product1
load(file = "all_active_clinical_entities.Rdata")

In [4]:
### Number of disorders
indice_list=1
disorders=list()
for (i in 1:length(all_active_clinical_entities))
  if (all_active_clinical_entities[[i]]$DisorderGroup$Name$text=='Disorder')
  {
    disorders[[indice_list]]=all_active_clinical_entities[[i]]
    indice_list=indice_list+1
  }
number_disorders=length(disorders)
number_disorders

[1] 6227

In [5]:
# We set the working directory
setwd('/home/jovyan/xml/')

In [6]:
suppressWarnings({
# We can now work on the genetic datas

# We open the xml "product3_156_en_jdbor_evo.xml" that contains the genetic disorders
xmlfiles = list.files(pattern='product3_156')
xmldoc=xmlParse(file=xmlfiles)
xml_156=xmlToList(xmldoc)

# We get the Classification information
prod_genetic=as.list(unlist(xml_156$ClassificationList))
# Each element in the list is a disorder
    })

In [7]:
# We get the OrphaCode and the DisorderType for each disorder
disordertype = list()
orpha = list()
indice_orpha = 1
indice_disordertype = 1
for (i in 1:length(prod_genetic))
  if (grepl("Disorder.OrphaCode", names(prod_genetic[i]))) {
    orpha[[indice_orpha]] = data.frame(orpha = prod_genetic[i])
    names(orpha[[indice_orpha]]) = c('orpha')
    indice_orpha = indice_orpha + 1
  } else if (grepl("DisorderType..attrs.id", names(prod_genetic[i]))) {
    disordertype[[indice_disordertype]] = data.frame(disordertype = prod_genetic[i])
    names(disordertype[[indice_disordertype]]) = c('disordertype')
    indice_disordertype = indice_disordertype + 1
  } 

disordertype = rbindlist(disordertype)
orpha = rbindlist(orpha)

In [8]:
# We only keep Disorders
disorders = cbind.data.frame(orpha, disordertype)
disorders = filter(disorders, disordertype == "21429" |
                     disordertype == "21401" |
                     disordertype == "21415" |
                     disordertype == "21394" |
                     disordertype == "21422" |
                     disordertype == "21408")

disorders = filter(disorders, !duplicated(disorders))

In [9]:
# Then we finaly get the number of genetic disorders
number_of_active_genetic_disorders=nrow(disorders)
percentage_of_active_genetic_disorders=number_of_active_genetic_disorders/number_disorders*100
print(paste0("The number of genetic disorders is ", round(number_of_active_genetic_disorders, digit=2)))
print(paste0("The percentage of genetic disorders is ", round(percentage_of_active_genetic_disorders, digit=2), "%"))

[1] "The number of genetic disorders is 4521"
[1] "The percentage of genetic disorders is 72.6%"


In [10]:
# And the number of non genetic disorders
number_of_non_active_genetic_disorders=number_disorders - number_of_active_genetic_disorders
percentage_of_non_active_genetic_disorders=number_of_non_active_genetic_disorders/number_disorders*100
print(paste0("The number of non-genetic disorders is ", number_of_non_active_genetic_disorders))
print(paste0("The percentage of non-genetic disorders is ", round(percentage_of_non_active_genetic_disorders, digit=2), "%"))

[1] "The number of non-genetic disorders is 1706"
[1] "The percentage of non-genetic disorders is 27.4%"
